In [ ]:
import numpy as np
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/RL/new/states.data', 'rb') as filehandle:
    # read the data as binary data stream
    STATES = pickle.load(filehandle)

with open('/content/drive/MyDrive/RL/new/transition_prob.data', 'rb') as filehandle:
    # read the data as binary data stream
    transition_probability = pickle.load(filehandle)
with open('/content/drive/MyDrive/RL/new/rewards.data', 'rb') as filehandle:
    # read the data as binary data stream
    REWARDS = pickle.load(filehandle)

In [ ]:
STATES = np.asarray(STATES) 
STATES_SPACE_SIZE = len(STATES)
ACTION_SPACE_SIZE = 9

In [ ]:
Q = np.ones((STATES_SPACE_SIZE, ACTION_SPACE_SIZE))

In [ ]:
beta = 0.2
gamma = 0.1
epsilon = 0.9
s_i = STATES[0].copy()
wd = 0
while True:
  Q_temp = Q.copy()
  a = np.where(s_i==0)[0]
  if len(a) == 0:
    a = range(9)
  
  a_method = np.random.choice([0 ,1],1, p = np.array([epsilon, 1-epsilon]))[0]
  ind = np.where((STATES==s_i).all(-1))[0][0] # index of current state
  
  if a_method == 0:
    ttt = np.argmax(Q[ind])
    if s_i[ttt]!=0:
      a = np.random.choice(a)
    else:
      a = ttt
  elif a_method == 1:
    # choosing action uniformly
    # getting actions possible from current state then chosing one of them uniformly
    a = np.random.choice(a)
  print('ind',ind)
  print('action',a)
  print(np.sum(transition_probability[ind][a]))
  ind2 = np.random.choice(np.arange(STATES_SPACE_SIZE),1,p= transition_probability[ind][a])[0]
  R = REWARDS[ind2]
  maxb = np.max(Q[ind2])
  Q[ind][a] += beta*(R + gamma*(maxb) - Q[ind,a])
  s_i = STATES[ind2].copy()
  print(np.max(np.absolute(Q_temp - Q) ) )
  if np.max(np.absolute(Q_temp - Q))<0.0001:
    wd+=1
  else:
    wd = 0
  if wd == 10:
    break


Streaming output truncated to the last 5000 lines.
1.0
0.0
ind 2616
action 1
1.0
0.00014212708806304875
ind 0
action 6
1.0
0.010796250033621835
ind 54
action 7
1.0
3.772449019834312e-11
ind 781
action 8
1.0
0.0
ind 2684
action 2
1.0
6.0688582990339746e-05
ind 0
action 6
1.0
0.020197375545677687
ind 52
action 4
1.0
0.28834375576347604
ind 649
action 2
1.0
0.0
ind 2595
action 5
1.0
0.00013065980398889454
ind 0
action 6
1.0
0.010391025321272562
ind 52
action 4
1.0
0.23067500461078083
ind 627
action 2
1.0
0.0
ind 2576
action 5
1.0
0.0006295127683779711
ind 0
action 6
1.0
0.03175645574373309
ind 51
action 7
1.0
1.3366893816516576
ind 778
action 0
0.99999994
7.411295882775448e-06
ind 981
action 8
1.0
4.263256414560601e-14
ind 2789
action 5
1.0
0.0008983898494386439
ind 0
action 4
1.0
0.01980541116656076
ind 35
action 8
1.0
1.7404427747891305e-06
ind 677
action 0
1.0
0.0
ind 2463
action 3
1.0
0.00045532314182088196
ind 0
action 4
1.0
0.0158405024872319
ind 33
action 6
1.0
4.0004902940538045e-

KeyboardInterrupt: ignored

In [ ]:
pi_star = [0]*len(STATES)
for i in range(3381):
  possible_action = np.where(STATES[i]==0)[0].tolist()
  d = {}
  if len(possible_action)>0:
    for ac in possible_action:
      d[ac] = Q[i][ac]
    pi_star[i] = max(d, key=d.get)

In [ ]:
with open('/content/drive/MyDrive/RL/new/pi_star_SARSA.data', 'wb') as filehandle:
    # read the data as binary data stream
     pickle.dump(pi_star,filehandle)

In [ ]:
pb = np.array([ep,1-ep])

In [ ]:
a_method = np.random.choice([0 ,1],1, p = pb)[0]
print(a_method)

1


In [ ]:
a_method

0

In [ ]:
#import modules
# import pygame
# import numpy as np
# import pickle
import itertools
# from pygame.locals import *
# np.seterr(divide='ignore', invalid='ignore')
'''
pygame.init()

screen_height = 300
screen_width = 300
line_width = 6
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Tic Tac Toe')



#define colours
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)

#define font
font = pygame.font.SysFont(None, 40)
'''
#define variables
clicked = False
player = 1
pos = (0,0)
markers = []
game_over = False
winner = 0

#setup a rectangle for "Play Again" Option
# again_rect = Rect(screen_width // 2 - 80, screen_height // 2, 160, 50)

#create empty 3 x 3 list to represent the grid
for x in range (3):
	row = [0] * 3
	markers.append(row)


'''
def draw_board():
	bg = (255, 255, 210)
	grid = (50, 50, 50)
	screen.fill(bg)
	for x in range(1,3):
		pygame.draw.line(screen, grid, (0, 100 * x), (screen_width,100 * x), line_width)
		pygame.draw.line(screen, grid, (100 * x, 0), (100 * x, screen_height), line_width)

def draw_markers():
	x_pos = 0
	for x in markers:
		y_pos = 0
		for y in x:
			if y == 1:
				pygame.draw.line(screen, red, (x_pos * 100 + 15, y_pos * 100 + 15), (x_pos * 100 + 85, y_pos * 100 + 85), line_width)
				pygame.draw.line(screen, red, (x_pos * 100 + 85, y_pos * 100 + 15), (x_pos * 100 + 15, y_pos * 100 + 85), line_width)
			if y == -1:
				pygame.draw.circle(screen, green, (x_pos * 100 + 50, y_pos * 100 + 50), 38, line_width)
			y_pos += 1
		x_pos += 1	

'''
def check_game_over(markers = markers):
	global game_over
	global winner

	x_pos = 0
	for x in markers:
		#check columns
		if sum(x) == 3:
			winner = 1
			game_over = True
		if sum(x) == -3:
			winner = 2
			game_over = True
		
		#check rows
		if markers[0][x_pos] + markers [1][x_pos] + markers [2][x_pos] == 3:
			winner = 1
			game_over = True
		if markers[0][x_pos] + markers [1][x_pos] + markers [2][x_pos] == -3:
			winner = 2
			game_over = True
		x_pos += 1

	#check cross
	if markers[0][0] + markers[1][1] + markers [2][2] == 3 or markers[2][0] + markers[1][1] + markers [0][2] == 3:
		winner = 1
		game_over = True
	if markers[0][0] + markers[1][1] + markers [2][2] == -3 or markers[2][0] + markers[1][1] + markers [0][2] == -3:
		winner = 2
		game_over = True

	#check for tie
	if game_over == False:
		tie = True
		for row in markers:
			for i in row:
				if i == 0:
					tie = False
		#if it is a tie, then call game over and set winner to 0 (no one)
		if tie == True:
			game_over = True
			winner = 0


'''
def draw_game_over(winner):

	if winner != 0:
		end_text = "Player " + str(winner) + " wins!"
	elif winner == 0:
		end_text = "You have tied!"

	end_img = font.render(end_text, True, blue)
	pygame.draw.rect(screen, green, (screen_width // 2 - 100, screen_height // 2 - 60, 200, 50))
	screen.blit(end_img, (screen_width // 2 - 100, screen_height // 2 - 50))

	again_text = 'Play Again?'
	again_img = font.render(again_text, True, blue)
	pygame.draw.rect(screen, green, again_rect)
	screen.blit(again_img, (screen_width // 2 - 80, screen_height // 2 + 10))

'''
def check_game_over_2(x):
		y = x.copy()
		y = np.array(y).reshape(3,3)
		y[np.where(y==2)] = -1
		# check for game over
		game_over = False
		check_game_over(y)

''' Defining STATE SPACE and required parameters'''
'''
states_0 = [[0]*9]

states_1 = []
board_pos = [i for i in range(9)]
for i in board_pos:
  for j in board_pos:
    if i!=j:
      x = [0]*9
      x[i] = 1
      x[j] = 2
      states_1.append(x)

# states_2
states_2 = []
for s in states_1:
  indices = [i for i, x in enumerate(s) if x == 1 or x == 2]
  board_pos = [i for i in range(9)]
  board_pos = list(set(board_pos) - set(indices))
  for i in board_pos:
    for j in board_pos:
      if i!=j:
        x = s.copy()
        x[i] = 1
        x[j] = 2
        if x not in states_2:
          states_2.append(x)

# states_3
TERMINAL_STATES = []
states_3 = []
for s in states_2:
  indices = [i for i, x in enumerate(s) if x == 1 or x == 2]
  board_pos = [i for i in range(9)]
  board_pos = list(set(board_pos) - set(indices))
  # print(len(board_pos))
  for i in board_pos:
    for j in board_pos:
      if i!=j:
        x = s.copy()
        # print(x,i,j)
        x[i] = 1
        # y = x.copy()
        # y = np.array(y).reshape(3,3)
        # y[np.where(y==2)] = -1
        # # check for game over
        game_over = False
        check_game_over_2(x)
        # if game is over then state_3.append(x) else
        if(game_over == True):
          # print("matrix\n",y)
          if x not in TERMINAL_STATES:
            TERMINAL_STATES.append(x)
            break
        else:
          x[j] = 2
          if x not in states_3:
            states_3.append(x)

# states_4
states_4 = []
# check for the
for s in states_3:
  indices = [i for i, x in enumerate(s) if x == 1 or x == 2]
  board_pos = [i for i in range(9)]
  board_pos = list(set(board_pos) - set(indices))
  for i in board_pos:
    for j in board_pos:
      if i!=j:
        x = s.copy()
        x[i] = 1
        x[j] = 2
        if x not in states_4:
          states_4.append(x)
'''
'''
with open('states.data', 'rb') as filehandle:
    # read the data as binary data stream
    STATES = pickle.load(filehandle)

# STATES = states_0+states_1+states_2+states_3+states_4+TERMINAL_STATES
ACTION_SPACE = [i for i in range(9)]

STATE_SPACE_SIZE = len(STATES)
ACTION_SPACE_SIZE = len(ACTION_SPACE)

'''

'''
transition_probability = np.array([[[0.0]*STATE_SPACE_SIZE]*ACTION_SPACE_SIZE]*STATE_SPACE_SIZE)


def generate_transition_prob():
  for i in range(STATE_SPACE_SIZE):
  	if STATES[i] in TERMINAL_STATES:
  		continue
  	for j in range(ACTION_SPACE_SIZE):
  		for k in range(STATE_SPACE_SIZE):
  			bd = STATES[i]
  			bd_2 = STATES[k]
  			if(bd[j] == 0 and bd_2[j]==1):
  				a = bd.copy()
  				a[j] = 1
  				game_over=False
  				check_game_over_2(a)
  				if game_over==True:
  					t_index = np.where((STATES==a).all(-1))[0][0]
  					transition_probability[i][j][t_index]=1.0
  				else:
  					check = np.subtract(bd_2,a)
  					if(np.sum(check)==2 and np.min(check)==0 and np.max(check)==2):
  						transition_probability[i][j][k] = np.random.randint(1,100,size=1)[0]


generate_transition_prob()

for i in range(STATE_SPACE_SIZE):
  for j in range(ACTION_SPACE_SIZE):
  	prob_sum = np.sum(transition_probability[i][j])
  	if prob_sum>0:
	    transition_probability[i][j] = transition_probability[i][j]/prob_sum
	    # print("sum=",np.sum(transition_probability[i][j]))
# print(transition_probability[0][2])

'''
'''
STATES = np.asarray(STATES) 
with open('transition_prob.data', 'rb') as filehandle:
    # read the data as binary data stream
    transition_probability = pickle.load(filehandle)
##################################################

with open('p_policy_iteration.data', 'rb') as filehandle:
    # read the data as binary data stream
    pi_star = pickle.load(filehandle)
'''
#main loop
win_count,tie_count,loss_count = 0,0,0
run = True
current_state = STATES[0]
ind = 0
while run:
	#draw board and markers first
	'''
	draw_board()
	draw_markers()
	
	#handle events
	for event in pygame.event.get():
		# print(markers)
		#handle game exit
		if event.type == pygame.QUIT:
			run = False
		'''
		#run new game
	if game_over == False:
		#check for mouseclick

		# if event.type == pygame.MOUSEBUTTONDOWN and clicked == False:
			# clicked = True
		# if event.type == pygame.MOUSEBUTTONUP and clicked == True:
		# clicked = False
		# pos = pygame.mouse.get_pos()
		# cell_x = pos[0] // 100 # cell_x = pos[0]
		# cell_y = pos[1] // 100 # cell_y = pos[1]
		# print("cell_x=",cell_x)
		# print("cell_y=",cell_y)
		# y = np.array(markers.copy())
		# y = y.reshape(1,-1)
		# y[np.where(y==-1)] = 2
		# y = y.tolist()[0]

		actionn = pi_star[ind]
		if markers[actionn//3][actionn%3] == 0:
			markers[actionn//3][actionn%3] = 1
			# print("actionn",actionn)
			game_over=False
			check_game_over(markers)
				# draw_markers()
			if game_over == False:

				# print("markers_1=",markers)

				# print("ind=",ind)	
				y = np.random.choice(np.arange(STATES_SPACE_SIZE),1,p= transition_probability[ind][actionn])[0]
				
				# print("index=",y)
				next_state = STATES[y].copy()
				current_state = next_state.copy()
				# print("current_state = ", current_state)
				next_state[np.where(next_state==2)] = -1
				markers = next_state.reshape(3,3).tolist()
				# print("markers_2-",markers)
				ind = np.where((STATES==current_state).all(-1))[0][0]
				# draw_markers()
				# player *= -1
			
				check_game_over(markers)

	#check if game has been won
	if game_over == True:
		if winner == 1:
			win_count+=1
		elif winner == 2:
			loss_count+=1
		else:
			tie_count+=1
		if win_count+loss_count+tie_count == 1000:
			break
		current_state = STATES[0]
		ind = 0
		'''
		draw_game_over(winner)
		#check for mouseclick to see if we clicked on Play Again
		if event.type == pygame.MOUSEBUTTONDOWN and clicked == False:
			clicked = True
		if event.type == pygame.MOUSEBUTTONUP and clicked == True:
			clicked = False
			pos = pygame.mouse.get_pos()
			if again_rect.collidepoint(pos):
				'''
				#reset variables
		game_over = False
		player = 1
		pos = (0,0)
		markers = []
		winner = 0
		#create empty 3 x 3 list to represent the grid
		for x in range (3):
			row = [0] * 3
			markers.append(row)

	#update display
	# pygame.display.update()

# pygame.quit()
print("win count:",win_count)
print("loss_count :",loss_count)
print("tie count:",tie_count)

win count: 976
loss_count : 0
tie count: 24


win count: 980
loss_count : 0
tie count: 20

win count: 975
loss_count : 0
tie count: 25


win count: 982
loss_count : 0
tie count: 18

win count: 971
loss_count : 0
tie count: 29


win count: 976
loss_count : 0
tie count: 24